<h1> Razak Olamide, KOLAWOLE

This project consider two algorithms, first, a modular computation algorithm for calculating the determinant of square matrices with integers coefficients.

The second, a divison-free algorithm for obtaining the characteristics polynomial and the determinant of square matrices whose coefficients are in a unital commutative ring $\mathcal{R}$ which is not an integral domain. 

<h1 style="color:blue"><strong> Modular Computation Algorithm
    

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)


Modular computation algorithm is a method centred around the methods of
division-free algorithm and fast algorithm like Gaussian elimination method.

For more info on this method check this book by Przemysław Koprowski: [lcm](http://www.pkoprowski.eu/lcm/lcm.pdf)

This method compute the determinant of integral matrices using the following
1. We define a function for finding the determinants of a matrix\
   a. Guassian elimination method (GEM)  (This steps require first changing the domain to field of rational)\
   b. Expansion method
2. We define a function for solving linear congruence equation (Chinese remainder theorem)

3. Finally, we define the main function
* One of the key steps in this function is casting the result from step 2 into field of       rationals, this is because GEM only works with field of rational
   

<p style="color:red"><strong> It is worth stating that our algorithm only works on matrices with integer coefficients

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

<h1 style="color:blue"><strong> We compute using Gaussian Elimination method 

In [57]:
def gaussian_elimination_det(matrix):
    """
    Compute the determinant of a matrix using GEM.

    Parameters:
        - M: Square matrix whose coefficents are in QQ

    Returns:
         Determinant of matrice
    """
    m = matrix.nrows()
    n = matrix.ncols()
    A = Matrix(matrix) # Make a copy of the matrix to avoid modifying the original
    # Initialize determinant to 1
    det = 1
    # Forward Elimination
    for i in range(min(m, n)):
        # Find pivot row
        pivot_row = i
        for k in range(i + 1, m):
            if abs(A[k, i]) > abs(A[pivot_row, i]):
                pivot_row = k
        if A[pivot_row, i] == 0:
            return 0  # If a pivot is zero, determinant is zero
        if pivot_row != i:
            # Swap rows if necessary
            A.swap_rows(i, pivot_row)
            # Swap changes sign of determinant
            det = -det
        det *= A[i, i]  # Multiply determinant by pivot
        # Eliminate elements below the pivot
        for k in range(i + 1, m):
            factor = A[k, i] / A[i, i]
            for j in range(i, n):
                A[k, j] -= factor * A[i, j]
    return det
 


In [58]:

def chinese_remainder_theorem(a, n):
    """
    Chinese Remainder Theorem algorithm to find x such that x ≡ a_i (mod n_i) for all i.

    Parameters:
        - a: List of remainders a_i
        - n: List of moduli n_i

    Returns:
        - x: The solution modulo the product of all moduli
    """
    N = prod(n)  # Calculate the product of all moduli
    x = 0
    for ai, ni in zip(a, n):
        Ni = N // ni  # Compute N_i = N / n_i
        x += ai * Ni * inverse_mod(Ni, ni)  # Compute the sum of a_i * N_i * inverse_mod(N_i, n_i)
    return x % N  # Return the solution modulo the product of all moduli




In [59]:
def modular_algorithm_for_determinant(M):
    """
    Compute the determinant of a matrix using modular arithmetic.

    Parameters:
        - M: Square matrix

    Returns:
        - det_mod: Determinant modulo the product of primes
    """
    for i in range(M.nrows()):
        for j in range(M.ncols()):
            assert M[i, j] in ZZ   # matrix coefficient must be integers
    n = M.nrows()       #number of rows of the determinants
    # Step 1: Find the Hadamard bound C and set the necessary condition on it
    C = max([abs(M[i, j]) for i in range(n) for j in range(n)])  
    Hadamard_bound = 2 * C^n * sqrt(n^n) # twice the Hadamard bound
# Step 2: Find odd primes p1, ..., pk
    primes = [p for p in range(2, ceil(Hadamard_bound)) if is_prime(p)]  
    product_of_primes = prod(primes)
    product_of_primes > Hadamard_bound
    if len(primes)%2==0:
        primes.append(next_prime(primes[-1]))
        product_of_primes = prod(primes)
# Step 3: Compute determinants mod pi using GEM 
    d1 = [Matrix(M).apply_map(lambda x: x % p) for p in primes]  
    d2 =[l.apply_map(lambda x: QQ(x)) for l in d1 ] # change the Domain of interest to rational
    d = [gaussian_elimination_det(v) for v in d2]
# Step 4: We use Chinese remainder theorem to obtain the solution of the systems obtained from step 3
    det_mod = chinese_remainder_theorem(d, primes)  
# Step 5: We compare the result obtained with our Hadamard bound
    if det_mod > product_of_primes / 2:
        det_mod = det_mod - product_of_primes
    return det_mod



<h1 style="color:blue"><strong> Example 1

In [60]:
# Example usage
M = Matrix([[3, 1, -2], [2, -3, -1], [1, 3, 0]])
det_mod = modular_algorithm_for_determinant(M)
print("The result of the computation of dterminant using modular algorithm")
print("Determinant of M:", det_mod)
%time

The result of the computation of dterminant using modular algorithm
Determinant of M: -10
CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs


<h1 style="color:blue"><strong> Example 2

In [61]:
M=matrix([[9,2,1],[5,-1,6],[4,0,-2]])
det_mod = modular_algorithm_for_determinant(M)
print("The result of the computation of dterminant using modular algorithm")
print("Determinant of M:", det_mod)

The result of the computation of dterminant using modular algorithm
Determinant of M: 90


<h1 style="color:blue"><strong> Comparing our result with the result of the in-buit function

In [62]:
M.det()==det_mod

True

<h1 style="color:blue"><strong> Alternatively, we compute using expansion method 

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)


In [63]:
def mydet(M):
    """
    This function returns the determinant of a matrix M computed
    recursively by expansion of the minors of the first column.
    
    """
    
    if M.ncols() != M.nrows(): # Check that M is a square matrix
        return "Not a square matrix, so the determinant is impossible to compute."
    else:
        n = M.ncols()
        if n == 1:
            return M[0, 0] # determinant of a 1x1 matrix
        else:
            D = 0
            for i in [0..n-1]:
                # Extract the submatrix excluding the ith row and first column
                N_rows = [j for j in [0..i-1]] + [j for j in [i+1..n-1]]
                N_cols = [j for j in [1..n-1]]
                N = M.matrix_from_rows_and_columns(N_rows, N_cols)
                # Compute the determinant recursively
                D += (-1)^i * M[i, 0] * mydet(N)
            return D


In [64]:
def modular_algorithm_for_determinant(M):
    """
    Compute the determinant of a matrix using modular arithmetic.

    Parameters:
        - M: Square matrix

    Returns:
        - det_mod: Determinant modulo the product of primes
    """
    for i in range(M.nrows()):
        for j in range(M.ncols()):
            assert M[i, j] in SR # the coefficients of the matrix M is any ring
    n = M.nrows()
    
    # Step 1: Find the Hadamard bound C and set the necessary condition on it
    C = max([abs(M[i, j]) for i in range(n) for j in range(n)])  
    # Step 2: Find odd primes p1, ..., pk
    Hadamard_bound = 2 * C^n * sqrt(n^n) #twice the Hadamard bound
    primes = [p for p in range(2, ceil(Hadamard_bound)) if is_prime(p)]  
    product_of_primes = prod(primes) # the product of primes
    product_of_primes > Hadamard_bound #the product of primes is twice the Hadamard C
    if len(primes)%2==0: # make sure the size of primes is odd
        primes.append(next_prime(primes[-1])) # Add one more prime to satisfy the condition
        product_of_primes = prod(primes) # Update the product of primes
        
    d = [mydet(Matrix(M).apply_map(lambda x: x % p)) for p in primes]  # Step 3: Compute determinants mod pi
    
    # Step 4: We use Chinese remainder theorem to obtain the solution of the systems obtained from step 3
    det_mod = chinese_remainder_theorem(d, primes)  
    if det_mod > product_of_primes / 2: # Step 5: compare the result obtained with our Hadamard bound
        det_mod = det_mod - product_of_primes
    else:
         det_mod=det_mod
    return det_mod



<h1 style="color:blue"><strong> Example 3

In [65]:
# For Example 
M = Matrix([[3, 1, -2], [2, -3, -1], [1, 3, 0]])
det_mod = modular_algorithm_for_determinant(M)
print("The result of the computation of dterminant using modular algorithm")
print("Determinant of M:", det_mod)
%time

The result of the computation of dterminant using modular algorithm
Determinant of M: -10
CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs


# Findings:


* Modular computation algorithm is indeed a fast computation algorith because of it's reliance on division algorithm i.e. Guassian elimination method

* when combined with minor method for determinants instead of GEM it takes longer time than usual

* The algorithm is really beautiful but my issue with it is that, it never tell us how much should the product of     primes 'm' be greater than the Hadamard bound.

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

<h1 style="color:blue"><strong> Samuelson Berkowitz algorithm

Samuelson Berkowitz algorithm is a divison-free algorithm for
obtaining the characteristics polynomial and the determinant of square matrices
whose coefficients are in a commutative ring $\mathcal{R}$ which is not an integral domain

* Toeplitz matrix is a very useful tool in this algorithm, so we first define a function that calculate the Toeplitz matrix of any square matrix which we would like to obtain its determinant and characteristics polynomial

* The advantage of Samuelson Berkowitz algorithm is that it can be used even when our domain of interest is not a field

For more info on this method check this book by Przemysław Koprowski: [lcm](http://www.pkoprowski.eu/lcm/lcm.pdf)

In [66]:
def toeplitz_matrix(M, k):
    
    """
    Compute the Toeplitz matrix of a square matrix.

    Parameters:
        - M: Square matrix

    Returns:
        - (n+2)x(n+1) Toeplitz matrix
    """
    
    # M_k is the kxk principal submatrix of M
    M_k = M.submatrix(0, 0, k, k)
    R_k = M[k, :k]  # Row vector below M_k
    C_k = M[:k, k]  # Column vector to the right of M_k
    m_kk = M[k, k]  # The element m_{k+1, k+1}
    
    # Initialize the Toeplitz matrix with -m_kk on the diagonal
    T_k = matrix(SR, k+2, k+1, lambda i, j: -m_kk if i == j else 0)
    
    # The first sub-diagonal is set to 1
    for i in [1, .., k+1]:
        T_k[i, i-1] = 1
    
    # Calculate the upper diagonal elements in T_k
    for i in [0, .., k]:  # loop over rows in T_k
        for j in [i+1, .., k]:  # loop over columns above the diagonal
            if j > i:
                # Calculate M_k to the power (j-i-1)
                if j-i-1 == 0:
                    M_k_i = matrix.identity(k)  # (M_k)^0 is the identity matrix
                else:
                    M_k_i = M_k^(j-i-1)  # M_k raised to the power (j-i-1)
                
                # Calculate -R_k * M_k_i * C_k for the respective entry and ensure scalar output
                tk = -R_k * M_k_i * vector(C_k)
                T_k[i, j] = tk[0]  # Extract the scalar value from the 1x1 matrix/product
                
    return T_k



In [67]:
def Samuelson_Berkowitz(M):
    """
    Compute the Characteristics Polynomial and the determinant of a matrix using Samuelson Identity.
    (determinant is the constant term in the characteristics polynomial)

    Parameters:
        - M: Square matrix with coefficients in any unital commutative ring R

    Returns:
        - Determinant and Characteristics Polynomial of a Square Matrix
    """
    n = M.ncols()  # find the number of column of the matrix M
    v = [-M[0][0], 1] #initialise column vector V with a value from M
    V = matrix(SR, 2, 1, v)
    v_vectors = []  #create a list
    for k in [1, .., n-1]:
        t_k = toeplitz_matrix(M, k)
        V1 = t_k * V  # Multiply current Toeplitz matrix by current vector V    
        # Update V to new V1
        V = V1 
        v_vectors.append(V)
        tk_matrices = r"T_{} = ".format(k) # This initializes tk_matrices as a string containing the prefix "T_k" 
    P.<x> = ZZ[]
    S = list(vector(v_vectors[-1])) # this create a column vector using the toeplitz matrix
    char_poly = P(S)   # create the characteristics 
    determinant = (-1)^n * S[0]  # the constant term of the polynomial
    return determinant, char_poly



<h1 style="color:blue"><strong> Example 4

In [68]:
# Define the matrix
M = matrix(ZZ, [[1, 0, 0], [0, 1, 0], [0, 0, 1]])
determinant, char_poly = Samuelson_Berkowitz(M)
show('The determinant:', determinant)
ola = r"Characteristics Polynomial: "
show(LatexExpr(ola), char_poly)

'The determinant:' 1

Characteristics Polynomial:  x^3 - 3*x^2 + 3*x - 1

<h1 style="color:blue"><strong> Example 5

In [69]:
M=matrix(ZZ, [[2,1,3],[5,-7,1],[3,0,-6]])
determinant, char_poly = Samuelson_Berkowitz(M)
show('The determinant:', determinant)
ola = r"Characteristics Polynomial: "
show(LatexExpr(ola), char_poly)

'The determinant:' 180

Characteristics Polynomial:  x^3 + 11*x^2 + 2*x - 180

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)


<h1 style="color:blue"><strong> Comparing our result with the result of the in-buit function

In [74]:
char_poly == M.charpoly()

True

In [75]:
determinant == M.det()

180 == 180

# Findings:

* The Samuelson-Berkowitz algorithm works well with matrices with coefficients in any unital commutative ring $\mathcal{R}$.

* It's slow just as was explained in the literature review

* It relies on the Samuelson Identity

* The Toeplitz matrix makes the computation very easy


<h1 style="color:blue"><strong> Thank you so much

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)